In [2]:
from gatv2 import GATV2

import torch
from torchdrug import datasets

dataset = datasets.ClinTox("~/molecule-datasets/")
#dataset = datasets.ZINC250k("~/molecule-datasets/", kekulize=True, atom_feature="symbol")

Constructing molecules from SMILES:   0%|          | 0/1484 [00:00<?, ?it/s]/Users/ca/Desktop/graph-generative-models/torchdg0.2.0/torchdrug/data/feature.py:42: UserWarning: Unknown value `*`
  warnings.warn("Unknown value `%s`" % x)
[20:33:34] Explicit valence for atom # 0 N, 5, is greater than permitted
/Users/ca/Desktop/graph-generative-models/torchdg0.2.0/torchdrug/data/feature.py:42: UserWarning: Unknown value `Tc`
  warnings.warn("Unknown value `%s`" % x)
/Users/ca/Desktop/graph-generative-models/torchdg0.2.0/torchdrug/data/feature.py:42: UserWarning: Unknown value `Fe`
  warnings.warn("Unknown value `%s`" % x)
/Users/ca/Desktop/graph-generative-models/torchdg0.2.0/torchdrug/data/feature.py:42: UserWarning: Unknown value `Al`
  warnings.warn("Unknown value `%s`" % x)
Constructing molecules from SMILES:   5%|▍         | 67/1484 [00:00<00:02, 669.37it/s]/Users/ca/Desktop/graph-generative-models/torchdg0.2.0/torchdrug/data/feature.py:42: UserWarning: Unknown value `Ca`
  warnings.wa

In [5]:
from typing import Optional
import torch
from torch import Tensor

from torch_geometric.utils.num_nodes import maybe_num_nodes


def degree(index: Tensor, num_nodes: Optional[int] = None,
           dtype: Optional[torch.dtype] = None) -> Tensor:
    N = maybe_num_nodes(index, num_nodes)
    out = torch.zeros((N, ), dtype=dtype, device=index.device)
    one = torch.ones((index.size(0), ), dtype=out.dtype, device=out.device)
    return out.scatter_add_(0, index, one)

In [18]:
import torch
from torchdrug import core, models, tasks
from torch import nn, optim
from collections import defaultdict
from torchdrug.layers import distribution

In [ ]:
salloc --account=su114-gpu -p gpu -N 1 -n 1 -c 42 --mem-per-cpu=3860 --gres=gpu:ampere_a100:1 --time=0:30:00

In [ ]:
model = models.RGCN(input_dim=dataset.node_feature_dim,
                    num_relation=dataset.num_bond_type,edge_input_dim=dataset.edge_feature_dim,
                    hidden_dims=[256, 256, 256], batch_norm=True)

In [53]:
deg = torch.zeros(10, dtype=torch.long)

model = PNA(input_dim=dataset.node_feature_dim,
                  hidden_dim=64, num_layer=3,
                  edge_input_dim=None,
                  num_relation=dataset.num_bond_type,
                  aggregators=['mean'],
                  scalers=['identity'],
                  deg=deg, batch_norm=True)

In [54]:
num_atom_type = dataset.num_atom_type
# add one class for non-edge
num_bond_type = dataset.num_bond_type + 1

node_prior = distribution.IndependentGaussian(torch.zeros(num_atom_type),
                                              torch.ones(num_atom_type))
edge_prior = distribution.IndependentGaussian(torch.zeros(num_bond_type),
                                              torch.ones(num_bond_type))
node_flow = models.GraphAF(model, node_prior, num_layer=1)
edge_flow = models.GraphAF(model, edge_prior, use_edge=True, num_layer=1)

In [55]:
task = tasks.AutoregressiveGeneration(node_flow, edge_flow,
                                      max_node=38, max_edge_unroll=12,
                                      task="plogp", criterion="ppo",
                                      reward_temperature=1, baseline_momentum=0.9,
                                      agent_update_interval=5, gamma=0.9)

In [ ]:
models.GearNet(input_dim=dataset.node_feature_dim,
                    num_relation=dataset.num_bond_type,
                    hidden_dims=[64, 64, 64], batch_norm=True)

In [ ]:
deg = torch.zeros(10, dtype=torch.long)
for data in dataset:
    graph = data['graph']
    d = degree(graph.edge_list[:, 1], num_nodes=graph.num_node, dtype=torch.long)
    deg += torch.bincount(d, minlength=deg.numel())

PNA(input_dim=dataset.node_feature_dim,
                  hidden_dim=64, num_layer=3,
                  edge_input_dim=dataset.edge_feature_dim,
                  num_relation=dataset.num_bond_type,
                  aggregators=['mean'],
                  scalers=['identity'],
                  deg=deg, batch_norm=True)

In [ ]:
from gsn import GSN
from gsn import prepare_GSN_dataset
prepare_GSN_dataset(dataset, max_cycle=8)

GSN(input_dim=dataset.node_feature_dim,
                  hidden_dim=64, num_layer=3,
                  edge_input_dim=dataset.edge_feature_dim,
                  num_relation=dataset.num_bond_type,
                  batch_norm=True)

In [ ]:
from torchdrug import core, tasks

model = GATV2(input_dim=dataset.node_feature_dim,
                    num_relation=dataset.num_bond_type,
                    hidden_dims=[256, 256, 256, 256], batch_norm=True)
task = tasks.GCPNGeneration(model, dataset.atom_types, max_edge_unroll=12,
                            max_node=38, criterion="nll")